In [1]:
reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:

%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from lxml import etree
import requests

In [3]:
headers = {
    'x-devtools-emulate-network-conditions-client-id': "c60c3ba6-26e8-45ce-9f9f-0b5521437cf6",
    'upgrade-insecure-requests': "1",
    'user-agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.143 Safari/537.36",
    'accept': "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    'referer': "http://isa.epfl.ch/imoniteur_ISAP/%21gedpublicreports.htm?ww_i_reportmodel=133685247",
    'accept-encoding': "gzip, deflate, sdch",
    'accept-language': "es-ES,es;q=0.8",
    'cookie': "LANGUE_LOGIN=fr",
    'cache-control': "no-cache"
}

data = requests.get("http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_i_reportModel=133685247", headers=headers)

soup = BeautifulSoup(data.content, 'html.parser')
queries = soup.find_all('select')
queries_map = {}
for s in queries:
    queries_map[s.get('name')] = {}
    map = queries_map[s.get('name')]
    options = s.find_all('option')
    for o in options:
        if not o.string:
            value = ""
        else:
            value = o.string
        map[value] = o.get('value')

print(queries_map)

{'ww_x_UNITE_ACAD': {'': 'null', 'Science et génie des matériaux': '944917', 'Systèmes de communication': '946228', 'Chimie et génie chimique': '246696', 'Management de la technologie': '946882', 'Sciences et technologies du vivant': '945901', 'Informatique': '249847', 'Cours de mathématiques spéciales': '943282', 'Génie civil': '942623', 'EME (EPFL Middle East)': '637841336', 'Génie électrique et électronique ': '943936', "Sciences et ingénierie de l'environnement": '942953', 'Mathématiques': '944590', 'Humanités digitales': '2054839157', 'Ingénierie financière': '120623110', 'Section FCUE': '1574548993', 'Physique': '945571', 'Architecture': '942293', 'Microtechnique': '945244', 'Génie mécanique': '944263'}, 'ww_x_PERIODE_ACAD': {'': 'null', '2013-2014': '213637754', '2007-2008': '978181', '2010-2011': '39486325', '2012-2013': '123456101', '2014-2015': '213637922', '2016-2017': '355925344', '2008-2009': '978187', '2009-2010': '978195', '2015-2016': '213638028', '2011-2012': '12345515

In [4]:
def getData(unite_acad, periode_acad, periode_pedago, semester_type):
    unite_acad_id = queries_map['ww_x_UNITE_ACAD'][unite_acad]
    periode_acad_id = queries_map['ww_x_PERIODE_ACAD'][periode_acad]
    periode_pedago_id = queries_map['ww_x_PERIODE_PEDAGO'][periode_pedago]
    semester_type_id = queries_map['ww_x_HIVERETE'][semester_type]
    url = "http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter"

    querystring = {"ww_b_list":"1",
                   "ww_i_reportmodel":"133685247",
                   "ww_c_langue":"",
                   "ww_i_reportModelXsl":"133685270",
                   "zz_x_UNITE_ACAD": unite_acad,
                   "ww_x_UNITE_ACAD": unite_acad_id,
                   "zz_x_PERIODE_ACAD": periode_acad,
                   "ww_x_PERIODE_ACAD":periode_acad_id,
                   "zz_x_PERIODE_PEDAGO":periode_pedago,
                   "ww_x_PERIODE_PEDAGO":periode_pedago_id,
                   "zz_x_HIVERETE":semester_type,
                   "ww_x_HIVERETE":semester_type_id,
                   "dummy":"ok"}

    headers = {
        'upgrade-insecure-requests': "1",
        'user-agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.143 Safari/537.36",
        'accept': "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
        'referer': "http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_i_reportModel=133685247",
        'accept-encoding': "gzip, deflate, sdch",
        'accept-language': "es-ES,es;q=0.8",
        'cookie': "LANGUE_LOGIN=fr",
        'cache-control': "no-cache",
        'postman-token': "65d96955-59bb-daa2-5136-807135f5b89a"
        }

    text = BeautifulSoup(requests.request("GET", url, headers=headers, params=querystring).text, 'html.parser')
    gps = list(text.find_all('table'))[2].find_all('tr')[1].td.a.get('onclick').split('ww_x_GPS=')[1].split("')")[0]
    payload = {'ww_x_GPS': int(gps),
               'ww_i_reportModel':133685247, 
               'ww_i_reportModelXsl': 133685271,
               'ww_x_UNITE_ACAD': unite_acad_id,
               'ww_x_PERIODE_ACAD': periode_acad_id,
               'ww_x_PERIODE_PEDAGO': periode_pedago_id,
               'ww_x_HIVERETE': semester_type_id
              }
    r = requests.get("http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.XLS", params=payload)
    return r.text

In [47]:
df_final = None
df_year = None

def addData(unite_acad, periode_acad, periode_pedago, semester_type, list_semesters):
    unite_acad_id = queries_map['ww_x_UNITE_ACAD'][unite_acad]
    periode_acad_id = queries_map['ww_x_PERIODE_ACAD'][periode_acad]
    periode_pedago_id = queries_map['ww_x_PERIODE_PEDAGO'][periode_pedago]
    semester_type_id = queries_map['ww_x_HIVERETE'][semester_type]

    
    soup = BeautifulSoup(getData(unite_acad,periode_acad,periode_pedago,semester_type), 'html.parser')
    rows = soup.find_all('tr')
    header_cols = rows[2].find_all('th')
    col_titles, data = [], {}
    
    for c in header_cols:
        data[c.string] = []
        col_titles.append(c.string)
    [col_titles.append(semestre[9:]) for semestre in list_semesters]
    for semestre in list_semesters:
        data[semestre[9:]] = [ float('nan') ] * ( len(rows) - 3)
    col_titles.append('enter')
    data['enter'] = None
    col_titles.append('exit')
    data['exit'] = None
    global df_final, df_year


    df = pd.DataFrame(columns=col_titles)
    if df_final is None:
        df_final = df.copy()
    if df_year is None:
        print("**********")
        df_year = df.copy() 


    for h in range(3, len(rows)):
        values = rows[h].find_all('td')
        for v in range(len(values)-2):
            data[header_cols[v].string].append(values[v].string)
        data[periode_pedago[9:]][h-3] =  True 
    df = pd.DataFrame.from_dict(data)

    return df



In [91]:
from IPython.display import display, HTML
def combineDataYear(new_data, old_data, semestre, year):
    print(semestre, year)
    for user in new_data['No Sciper'].tolist():
        if (user in old_data['No Sciper'].tolist()):
            #print(old_data[semestre[9:]].loc[old_data['No Sciper'] == user], "*******")
            old_data[semestre[9:]].loc[old_data['No Sciper'] == user] = True
            old_data['exit'].loc[old_data['No Sciper'] == user] = year
        else:
            new_data['enter'].loc[new_data['No Sciper'] == user] = year
            new_data['exit'].loc[new_data['No Sciper'] == user] = year
            old_data = pd.concat([old_data, new_data.loc[new_data['No Sciper'] == user]], ignore_index=True)
            #print(new_data['semestre 1'].loc[new_data['No Sciper'] == user])
    return old_data

def combineDataTotal(new_data, old_data):
    old_data = old_data.combine_first(new_data)
    for user in new_data['No Sciper'].tolist():
        #if user in old_data['No Sciper'].tolist():
        #    old_data.loc[old_data['No Sciper'] == user] = old_data.loc[old_data['No Sciper'] == user].combine_first(new_data.loc[new_data['No Sciper'] == user])
            
            
            #old_data.update(new_data, overwrite=True)
        #else:
        if user not in old_data['No Sciper'].tolist():
            old_data = pd.concat([old_data, new_data.loc[new_data['No Sciper'] == user]])
    return old_data

In [92]:
unite_acad = "Informatique"
periode_acad = list(queries_map['ww_x_PERIODE_ACAD'].keys())
periode_acad.remove('')
periode_pedago = []
for p in list(queries_map['ww_x_PERIODE_PEDAGO'].keys()):
    if 'Bachelor' in p and "b" not in p:
        periode_pedago.append(p)
periode_pedago = sorted(periode_pedago)
periode_acad = sorted(periode_acad)
df_final = None
df_year = None


for year in range(0,2):
    df_year = None
    for s in range(len(periode_pedago)):
        a = addData("Informatique", periode_acad[year], periode_pedago[s], "", periode_pedago)
        df_year = combineDataYear(a, df_year, periode_pedago[s], periode_acad[year])
    df_final = combineDataTotal(df_year, df_final)
display(df_final)


#df_final = df_final.combine_first(df_year)


**********
Bachelor semestre 1 2007-2008


/Users/nachoaguadoc/anaconda/lib/python3.5/site-packages/pandas/core/indexing.py:132: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


Bachelor semestre 2 2007-2008
Bachelor semestre 3 2007-2008
Bachelor semestre 4 2007-2008
Bachelor semestre 5 2007-2008
Bachelor semestre 6 2007-2008
**********
Bachelor semestre 1 2008-2009
Bachelor semestre 2 2008-2009
Bachelor semestre 3 2008-2009
Bachelor semestre 4 2008-2009
Bachelor semestre 5 2008-2009
Bachelor semestre 6 2008-2009


,Civilité,Ecole Echange,Filičre opt.,Mineur,No Sciper,Nom Prénom,Orientation Bachelor,Orientation Master,Spécialisation,Statut,Type Echange,enter,exit,semestre 1,semestre 2,semestre 3,semestre 4,semestre 5,semestre 6
0,Monsieur,None,NaN,NaN,169569,Arévalo Christian,NaN,NaN,NaN,Présent,None,2007-2008,2007-2008,True,True,NaN,NaN,NaN,NaN
1,Monsieur,None,NaN,NaN,174905,Aubelle Flavien,NaN,NaN,NaN,Présent,None,2007-2008,2007-2008,True,True,NaN,NaN,NaN,NaN
2,Monsieur,None,NaN,NaN,173922,Badoud Morgan,NaN,NaN,NaN,Présent,None,2007-2008,2007-2008,True,True,NaN,NaN,NaN,NaN
3,Monsieur,None,NaN,NaN,179406,Baeriswyl Jonathan,NaN,NaN,NaN,Présent,None,2007-2008,2007-2008,True,True,NaN,NaN,NaN,NaN
4,Monsieur,None,NaN,NaN,179428,Barroco Michael,NaN,NaN,NaN,Présent,None,2007-2008,2007-2008,True,True,NaN,NaN,NaN,NaN
5,Monsieur,None,NaN,NaN,179324,Belfis Nicolas,NaN,NaN,NaN,Présent,None,2007-2008,2007-2008,True,True,NaN,NaN,NaN,NaN
6,Monsieur,None,NaN,NaN,174597,Beliaev Stanislav,NaN,NaN,NaN,Présent,None,2007-2008,2007-2008,True,True,NaN,NaN,NaN,NaN
7,Monsieur,None,NaN,NaN,179449,Bindschaedler Vincent,NaN,NaN,NaN,Présent,None,2007-2008,2007-2008,True,True,NaN,NaN,NaN,NaN
8,Monsieur,None,NaN,NaN,178553,Bloch Marc-Olivier,NaN,NaN,NaN,Présent,None,2007-2008,2007-2008,True,True,NaN,NaN,NaN,NaN
9,Monsieur,None,NaN,NaN,179426,Bloch Remi,NaN,NaN,NaN,Présent,None,2007-2008,2007-2008,True,True,NaN,NaN,NaN,NaN


In [96]:
df_final['semestre 1'].loc[df_year['No Sciper'] == "169569"]

96    True
Name: semestre 1, dtype: bool